In [17]:
using ITensors

In [18]:
Lx, Ly = 2, 2;
nTot = Lx*Ly*2;
t = 1.0;
V = 10;

In [19]:
function site2Idx(ix, iy, iu)
    return Lx*Ly*(iu) + (ix)*Ly + iy + 1
end

sites = siteinds("Fermion", Lx*Ly*2; conserve_qns=true);

os = OpSum();
for ix in 0:Lx-1
    for iy in 0:Ly-1
        os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, iy, 1)
        os += -t, "Cdag", site2Idx(ix, iy, 1), "C", site2Idx(ix, iy, 0)
        os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx((ix+1)%Lx, iy, 1)
        os += -t, "Cdag", site2Idx((ix+1)%Lx, iy, 1), "C", site2Idx(ix, iy, 0)
        os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, (iy+1)%Ly, 1)
        os += -t, "Cdag", site2Idx(ix, (iy+1)%Ly, 1), "C", site2Idx(ix, iy, 0)
        os += V, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, iy, 1)
        os += V, "N", site2Idx(ix, iy, 0), "N", site2Idx((ix+1)%Lx, iy, 1)
        os += V, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, (iy+1)%Ly, 1)
    end
end

state = ["Emp" for n in 1:Lx*Ly*2]
for i in 1:Lx*Ly
    state[i] = "Occ"
end

psi0 = randomMPS(sites, state, 10);

nsweeps = 6
maxdim = [50, 100, 200, 400, 800, 800]
cutoff = [1E-12];

In [20]:
function eFromV(v)
    os = OpSum();
    for ix in 0:Lx-1
        for iy in 0:Ly-1
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, iy, 1)
            os += -t, "Cdag", site2Idx(ix, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx((ix+1)%Lx, iy, 1)
            os += -t, "Cdag", site2Idx((ix+1)%Lx, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, (iy+1)%Ly, 1)
            os += -t, "Cdag", site2Idx(ix, (iy+1)%Ly, 1), "C", site2Idx(ix, iy, 0)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx((ix+1)%Lx, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, (iy+1)%Ly, 1)
        end
    end
    H = MPO(os, sites);
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff, outputlevel = 0);
    d = fill(0.0, Lx*Ly*2)
    for j in 1:Lx*Ly*2
        orthogonalize!(psi, j)
        psidag_j = dag(prime(psi[j], "Site"))
        d[j] = scalar(psidag_j * op(sites, "N", j) * psi[j])
    end
    println("V=", v, " E=", energy-0.375*nTot*v, "\ndensity=", d)
end

eFromV (generic function with 1 method)

In [21]:
for v in 0.1:0.1:2.0
    eFromV(v)
end

V=0.1 E=-6.077158208310033
density=[0.5000000000000002, 0.5000000000000002, 0.5000000000000003, 0.5, 0.49999999999999967, 0.4999999999999999, 0.49999999999999956, 0.4999999999999996]
V=0.2 E=-6.158831726627021
density=[0.5000000000000003, 0.5000000000000002, 0.5000000000000002, 0.5, 0.4999999999999993, 0.49999999999999967, 0.49999999999999956, 0.4999999999999998]
V=0.3 E=-6.245325367750455
density=[0.5000000000000007, 0.5000000000000006, 0.5000000000000007, 0.5000000000000003, 0.49999999999999883, 0.4999999999999991, 0.4999999999999991, 0.49999999999999895]
V=0.4 E=-6.336949272411735
density=[0.5000000000000007, 0.5000000000000007, 0.5000000000000008, 0.5000000000000007, 0.49999999999999933, 0.49999999999999944, 0.499999999999999, 0.4999999999999993]
V=0.5 E=-6.434013869095791
density=[0.5000000000000006, 0.5000000000000004, 0.5000000000000009, 0.5000000000000012, 0.49999999999999956, 0.4999999999999999, 0.49999999999999983, 0.4999999999999998]
V=0.6 E=-6.5368237609266595
density=[0.50